In [1]:
import sys
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from time import time

Using TensorFlow backend.


In [2]:
start_time_0 = time()

### Load file

In [3]:
lyrics = open('BSB.txt').read()
lyrics = lyrics.lower()

In [4]:
len(lyrics)

36518

### Convert char to num

In [5]:
chars = [i for i in sorted(set(lyrics))]

In [6]:
char_num = dict((char, num) for num, char in enumerate(chars))
num_char = dict((num, char) for num, char in enumerate(chars))

In [7]:
lyrics_num = [char_num[i] for i in lyrics]


In [11]:
alphabet = len(char_num)

### create sequence

In [8]:
def chop_to_sequence(seq, lyrics_num):
    """
    chop lyrics_num into segments with length seq
    return list of segments
    """
    lyrics_arr = np.asarray(lyrics_num)
    lyrics_arr = lyrics_arr / float(alphabet)
    segments = []
    next_char = []
    for i in range(0, len(lyrics_num)-seq):
        segment = lyrics_arr[i:i+seq]
        segments.append(segment)
        next_char.append(lyrics_num[i+seq])

    print("segment length:", seq)
    print('number of segments:', len(segments))
    print("chars in lyrics:", len(lyrics))
    print("")
    
    segments = np.reshape(segments, (len(segments),seq,1))
    next_char = np_utils.to_categorical(next_char)
    
    return segments, next_char
    

In [9]:
def print_time(start_time):
    print((time()-start_time)/60)

In [12]:
start_time = time()
X_all, y_all = chop_to_sequence(30, lyrics_num)
print_time(start_time)


segment length: 30
number of segments: 36488
chars in lyrics: 36518

0.0016046524047851562


### LSTM model

In [13]:
def predict_next_n(n):
    """
    predict next n char from random seed
    """
    start = np.random.randint(0, len(X))
    seed = lyrics[start:start+X.shape[1]]
    pattern = X[start]
    
    chars=[]
    for i in range(n):
        pred_num = model.predict_classes(np.reshape(pattern,(1,pattern.shape[0],1)), verbose=False)
        #pred_num = np.argmax(pred_arr)
        pred_char = num_char[pred_num[0]]
        
        chars.append(pred_char)
        pattern = np.append(pattern, pred_num/float(alphabet))
        pattern = pattern[1:]
        #print(pred_arr, pred_num, pred_char, pattern)
    print("Seed:", seed)
    print("Generated:", "".join(chars))
    

### All data

In [17]:
X=X_all[:1000]
y=y_all[:1000]

In [28]:
model = Sequential()
model.add(LSTM(alphabet, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(alphabet))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))

In [29]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 30, 39)            6396      
_________________________________________________________________
dropout_3 (Dropout)          (None, 30, 39)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 39)                12324     
_________________________________________________________________
dropout_4 (Dropout)          (None, 39)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 39)                1560      
Total params: 20,280.0
Trainable params: 20,280.0
Non-trainable params: 0.0
_________________________________________________________________


In [41]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


In [21]:
from keras.optimizers import Adam

In [30]:
adam_lr = Adam(lr=0.01)

In [31]:
model.compile(loss='categorical_crossentropy', optimizer=adam_lr, metrics=['accuracy'], )
start_time = time()
model.fit(X, y, epochs=100, batch_size=64, 
          #callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 4s - loss: 3.2902 - acc: 0.1350 - val_loss: 3.0595 - val_acc: 0.1600

KeyboardInterrupt: 

In [24]:
model.compile(loss='categorical_crossentropy', optimizer=adam_lr, metrics=['accuracy'], )
start_time = time()
model.fit(X, y, epochs=100, batch_size=64, 
          #callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 4s - loss: 3.1237 - acc: 0.1363 - val_loss: 3.0506 - val_acc: 0.1600

KeyboardInterrupt: 

In [24]:
model.compile(loss='categorical_crossentropy', optimizer=adam_lr, metrics=['accuracy'], )
start_time = time()
model.fit(X, y, epochs=100, batch_size=64, 
          #callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 4s - loss: 3.1237 - acc: 0.1363 - val_loss: 3.0506 - val_acc: 0.1600

KeyboardInterrupt: 

In [32]:
adam_lr = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam_lr, metrics=['accuracy'], )
start_time = time()
model.fit(X, y, epochs=25, batch_size=64, 
          #callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/25
800/800 [==============================] - 4s - loss: 2.8999 - acc: 0.1638 - val_loss: 2.9163 - val_acc: 0.1600

In [36]:
adam_lr = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam_lr, metrics=['accuracy'], )
start_time = time()
model.fit(X, y, epochs=250, batch_size=64, 
          #callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/250
800/800 [==============================] - 4s - loss: 2.7823 - acc: 0.1762 - val_loss: 2.7620 - val_acc: 0.1800

In [37]:

model.fit(X, y, epochs=250, batch_size=64, 
          #callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/250
800/800 [==============================] - 3s - loss: 1.8015 - acc: 0.4425 - val_loss: 1.3297 - val_acc: 0.6650

In [38]:

model.fit(X, y, epochs=250, batch_size=64, 
          #callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/250
800/800 [==============================] - 2s - loss: 1.2782 - acc: 0.5900 - val_loss: 0.6424 - val_acc: 0.8500

In [39]:

model.fit(X, y, epochs=100, batch_size=64, 
          #callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 3s - loss: 1.0288 - acc: 0.6613 - val_loss: 0.3466 - val_acc: 0.9300

In [40]:

model.fit(X, y, epochs=100, batch_size=64, 
          #callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 3s - loss: 0.9735 - acc: 0.6763 - val_loss: 0.2877 - val_acc: 0.9600

In [42]:

model.fit(X, y, epochs=1, batch_size=64, 
          callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 3s - loss: 0.8792 - acc: 0.7150 - val_loss: 0.2300 - val_acc: 0.9650


In [22]:
print_time(start_time)

40.12507123549779


In [43]:
predict_next_n(100)

Seed: thing i do
(is for you baby)
i
Generated:  wanna be with you
(oh with you

everybody say yeue
aand everythinw io i lannn to wnathe
io's i mone


In [44]:
for i in range(10):
    predict_next_n(30)


Seed: eah
everybody swing

wish you'
Generated: c come and setese you l ce s w
Seed: i wanna be with you
it's crazy
Generated:  but it's true
(you know it's 
Seed: at to do
i wanna be with you


Generated: and i'll be waiting
until you 
Seed: wanna be with you

and i'll be
Generated:  waiting
until you face the tr
Seed: by)
i wanna be with you
(oh wi
Generated: th you

everybody say yeue
aan
Seed: d
what's missing in me
i'm loo
Generated: king fou y ae
i'elke to know w
Seed: y
yeah
everybody swing

wish y
Generated: ou'c come and setese you l ce 
Seed: ting
until you face the truth

Generated: when the light is fading
you k
Seed: again
yeah
everybody swing
yea
Generated: h
everybody seah
everybody swi
Seed: all the things i've left behin
Generated: 
yhat's in seue yo tha linn
io


In [45]:

model.fit(X, y, epochs=200, batch_size=64, 
          #callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/200
800/800 [==============================] - 3s - loss: 0.8424 - acc: 0.7000 - val_loss: 0.2382 - val_acc: 0.9550

In [46]:

model.fit(X, y, epochs=1, batch_size=64, 
          callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 3s - loss: 0.7270 - acc: 0.7675 - val_loss: 0.1628 - val_acc: 0.9800


In [47]:

for i in range(10):
    predict_next_n(30)


Seed: thing i do
(is for you baby)
i
Generated:  wanna be with you
(oh with yo
Seed: i wanna be with you
it's crazy
Generated:  but it's true
(you know it's 
Seed: e with you
(oh with you)
every
Generated: body say he
eeee be with you


Seed:  do
(is for you baby)
i wanna 
Generated: be with you
(oh with you

ever
Seed: h
everybody swing

wish you'd 
Generated: come and set me free
so that i
Seed:  i do
i wanna be with you

i'd
Generated:  like to know yout yoat d lann
Seed: what to do
i wanna be with you
Generated: 

and i'll be waiting
until yo
Seed: azy but it's true
(you know it
Generated: 's true)
and everything i do
i
Seed:  it's true)
and everything i d
Generated: o
i manna to whet youngi i's m
Seed:  you
(oh with you)
everybody s
Generated: ay he
eeee be with you

andgn



In [48]:
start_time = time()
model.fit(X, y, epochs=50, batch_size=64, verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/50
800/800 [==============================] - 3s - loss: 0.7369 - acc: 0.7587 - val_loss: 0.1614 - val_acc: 0.9650

In [49]:

model.fit(X, y, epochs=1, batch_size=64, 
          callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 3s - loss: 0.7641 - acc: 0.7563 - val_loss: 0.1551 - val_acc: 0.9850


In [ ]:
print_time(start_time)

In [50]:
predict_next_n(100)

Seed: a be with you

and i'll be wai
Generated: ting
until you face the truth
when the light is fading
you know what i wanna do

i wanna be with you


In [51]:

model.fit(X, y, epochs=500, batch_size=64, 
          #callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/500
800/800 [==============================] - 3s - loss: 0.7410 - acc: 0.7662 - val_loss: 0.1498 - val_acc: 0.9800

In [52]:

model.fit(X, y, epochs=300, batch_size=64, 
          #callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/300
800/800 [==============================] - 3s - loss: 0.6233 - acc: 0.8000 - val_loss: 0.1092 - val_acc: 0.9800

In [53]:
model.fit(X, y, epochs=1, batch_size=64, 
          callbacks= callbacks_list, 
          verbose= True, validation_split=0.2)
for i in range(10):
    predict_next_n(30)


Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 3s - loss: 0.5470 - acc: 0.7950 - val_loss: 0.0602 - val_acc: 0.9900
Seed: e with you
(oh with you)
every
Generated: body say hey!
yeah
see you onc
Seed: looking for a sign
in the thin
Generated: gs you do
i wanna be with you

Seed: ith you
it's crazy but it's tr
Generated: ue
(you know it's true)
and ev
Seed: e again
yeah
everybody swing
y
Generated: eah
everybody
yeah
everybody s
Seed: e
(you know it's true)
and eve
Generated: rything i do
i wanna be with y
Seed:  with you
(oh with you)
everyb
Generated: ody say hey!
yeah
see you once
Seed: 
it's crazy but it's true
(you
Generated:  know it's true)
and everythin
Seed: ody
yeah
everybody swing

wish
Generated:  you'd come and set me free
so
Seed:  you

and i'll be waiting
unti
Generated: l you face the truth
when the 
Seed: 've left behind
what's missing
Generated:  in se
i'm looking for i sign



In [54]:
predict_next_n(1000)

Seed: ght is fading
you know what i 
Generated: wanna do

i wanna be with you
it's crazy but it's true
(you know it's true)
and everything i do
i wanna be with you

i'd like to know your policy
ihl i monn  ndwit  you 
thee be eeee
yo tou inye iou iewe ever to sei leena you
you yeu saue you it's erun you it's taaere
yo 
thei the thanh
in
so i segna yoiiio ihai i w iamen th whet t t di watr you
know what's wauh 
you 
and tre thu iths 
n's wanna you

ane whe true

nder toe than shin s saueueatg ind i iiliang yoath toat's mamenn that i can kie to whe te t's erany you
yntn i's lanna be with you

and w'wr yo
you

and i'likenbgyhwhaadi i'lign
iosr toen s me
ief seeth yoat saat tre you ieee
youe se i w era be wiaaa yo t's laeth you 
acd w waat  tha er wa 
ah fne yeu bnoa the tiue (ad  l'wiee oo fn wi t eeery but tt what s tageues yaai aail foe to she that'seseneos
s d lig nn th whth you ith irar so ses femza yea co i wlinng wou fo i wanna be with you

and i'll be waiting
until you face the tru